In [ ]:
from transformers import MarianMTModel, MarianTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments

import pandas as pd

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
splits = {'train': 'train.jsonl', 'validation': 'val.jsonl', 'test': 'test.jsonl'}
train = pd.read_json("hf://datasets/bentrevett/multi30k/" + splits["train"], lines=True)
val = pd.read_json("hf://datasets/bentrevett/multi30k/" + splits["validation"], lines=True)
test = pd.read_json("hf://datasets/bentrevett/multi30k/" + splits["test"], lines=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
train.info()
print()
val.info()
print()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29000 entries, 0 to 28999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   en      29000 non-null  object
 1   de      29000 non-null  object
dtypes: object(2)
memory usage: 453.2+ KB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1014 entries, 0 to 1013
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   en      1014 non-null   object
 1   de      1014 non-null   object
dtypes: object(2)
memory usage: 16.0+ KB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   en      1000 non-null   object
 1   de      1000 non-null   object
dtypes: object(2)
memory usage: 15.8+ KB


In [ ]:
print(train.head())

                                                  en  \
0  Two young, White males are outside near many b...   
1  Several men in hard hats are operating a giant...   
2    A little girl climbing into a wooden playhouse.   
3  A man in a blue shirt is standing on a ladder ...   
4           Two men are at the stove preparing food.   

                                                  de  
0  Zwei junge weiße Männer sind im Freien in der ...  
1  Mehrere Männer mit Schutzhelmen bedienen ein A...  
2  Ein kleines Mädchen klettert in ein Spielhaus ...  
3  Ein Mann in einem blauen Hemd steht auf einer ...  
4  Zwei Männer stehen am Herd und bereiten Essen zu.  


In [ ]:
model = "Helsinki-NLP/opus-mt-de-en"
tokenizer = MarianTokenizer.from_pretrained(model)

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/768k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
tokenized_train = tokenizer(list(train["de"]), text_target=list(train["en"]), max_length=128, truncation=True)
tokenized_val = tokenizer(list(val["de"]), text_target=list(val["en"]), max_length=128, truncation=True)
tokenized_test = tokenizer(list(test["de"]), text_target=list(test["en"]), max_length=128, truncation=True)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq

# Load the Pre-Trained Model
model = AutoModelForSeq2SeqLM.from_pretrained(model)

# Define the Data Collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

pytorch_model.bin:   0%|          | 0.00/298M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
import evaluate
import numpy as np

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   return metric.compute(predictions=predictions, references=labels)

In [ ]:
from datasets import Dataset

# Converted the tokenized data to Dataset
train_data_dict = {key: val for key, val in tokenized_train.items()}
val_data_dict = {key: val for key, val in tokenized_val.items()}

# Created Dataset objects
tokenized_train_dataset = Dataset.from_dict(train_data_dict)
tokenized_val_dataset = Dataset.from_dict(val_data_dict)


In [ ]:
# Define training arguments
training_args = Seq2SeqTrainingArguments(
    run_name="translation_german_to_english",
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    logging_dir='./logs',
    logging_steps=10,
    fp16=True,
    save_steps=500
)

# Define trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset= tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)




/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-14-bcbd791825dd>:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
